In [2]:
import requests
from parsel import Selector
import json
import os

In [46]:
import requests
from parsel import Selector
import json
import os


class GetBv():
    def __init__(self, bvid, page=1):
        '''
        bvid:视频号 eg:BV1hE411N7q2，str
        page: P几的视频，默认为1 int
        '''
        self.bvid = bvid
        self.page = page
        self.pg_dic, self.file_name, self.all_page = GetBv.get_name_pages(self)  # 视频名称表单、文件夹名称、总视频数
        # 表头
        self.headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:2.0.1) Gecko/20100101 Firefox/4.0.1',
                        'Referer': 'https://www.bilibili.com',}
    
    # 获取视频网址
    def get_video_audio_urls(self):
        url = f'https://www.bilibili.com/video/{self.bvid}?p={self.page}'
        r = requests.get(url)
        selector = Selector(text=r.text)
        # 视频连接
        video_audio = selector.xpath('/html/head/script[5]/text()').getall()
        video_dic = video_dic = json.loads(video_audio[0][video_audio[0].index('{'):])['data']['dash']['video'] # 找到{}才能loads
        video_url = video_dic[0]['baseUrl']
        # 音频连接
        audio = video_dic = json.loads(video_audio[0][video_audio[0].index('{'):])['data']['dash']['audio']
        audio_url = audio[0]['baseUrl']
        return video_url, audio_url
    
    # 获取视频名称表单并创建存放视频的文件夹
    def get_name_pages(self):
        # 获取名称表单
        url = f'https://www.bilibili.com/video/{self.bvid}'
        r_ng = requests.get(url)
        selector = Selector(r_ng.text)
        
        video_message = selector.xpath('/html/head/script[6]/text()').getall()
        video_message = json.loads(video_message[0][video_message[0].index('{'):video_message[0].index('};')+1])      # 找到{}才能loads
        
        all_page = video_message['videoData']['videos']    # 总页数/视频数
        page_name_dic = {}             # 页数对应的名称
        for x in video_message['videoData']['pages']:
            page_name_dic[x['page']] = x['part']
            
        # 创建文件夹
        file_name = video_message['videoData']['title']
        if os.path.exists(file_name) == False: # 判断是否存在该文件夹
            os.mkdir(file_name)
            
        return page_name_dic, file_name,all_page
        
    # 获取video.m4s和audio.m4s格式的音频和视频
    def get_mp4(self):
        # bv_name: 视频名称
        bv_name = self.pg_dic[self.page]
        path = f'{self.file_name}/{bv_name}'    # 视频路径
        print(f'{bv_name}.mp4')
        if os.path.exists(f'{path}.mp4'):  # 判断是否存在该视频
            print(f'已存在第{self.page}P视频——{bv_name}.mp4')
        else:
            video_url, audio_url = GetBv.get_video_audio_urls(self)
            # 视频
            r_video = requests.get(video_url, headers=self.headers)

            with open(f'{path}_video.m4s', 'wb') as f:
                f.write(r_video.content)
            # 音频
            r_video = requests.get(audio_url, headers=self.headers)
            with open(f'{self.file_name}/{bv_name}_audio.m4s', 'wb') as f:
                f.write(r_video.content)
            # 合并视频
            order = f'ffmpeg -i "{path}_video.m4s" -i "{path}_audio.m4s" -codec copy "{path}.mp4"'  # 加引号是为了解决路径不能包含空格 . 的问题
            os.system(order)
            # 删除原来的音频+视频.m4s文件
            os.remove(f"{path}_audio.m4s")
            os.remove(f"{path}_video.m4s")

            print(f'成功爬取,第{self.page}P视频——{bv_name}')

if __name__ == '__main__':
    bvid = 'BV1Pt411G7my'           # ----视频的id----
    getBv = GetBv(bvid)
    max_page = getBv.all_page        # 获取最大页数
    page_list = list(range(1, max_page+1))   # ---爬取第几P视频的列表-，记得加1--
    print(f'------------开始爬取------------\n 本次爬取第{page_list}P的视频')
    
    for page in page_list:
        getBv.page = page
        getBv.get_mp4()

------------开始爬取------------
 本次爬取第[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]P的视频
01 绪论.mp4
已存在第1P视频——01 绪论.mp4
02 后验分布的计算.mp4
已存在第2P视频——02 后验分布的计算.mp4
03 共轭先验分布的特点（一）.mp4
已存在第3P视频——03 共轭先验分布的特点（一）.mp4
04 共轭先验分布的特点（二）.mp4
已存在第4P视频——04 共轭先验分布的特点（二）.mp4
05 共轭先验分布的特点（三）.mp4
已存在第5P视频——05 共轭先验分布的特点（三）.mp4
06 超参数及其确定.mp4
已存在第6P视频——06 超参数及其确定.mp4
07 贝叶斯统计推断的条件方法.mp4
已存在第7P视频——07 贝叶斯统计推断的条件方法.mp4
08 参数的贝叶斯点估计（一）.mp4
已存在第8P视频——08 参数的贝叶斯点估计（一）.mp4
09 参数的贝叶斯点估计（二）.mp4
已存在第9P视频——09 参数的贝叶斯点估计（二）.mp4
10 参数的贝叶斯点估计（三）.mp4
已存在第10P视频——10 参数的贝叶斯点估计（三）.mp4
11 参数的贝叶斯点估计（四）.mp4
已存在第11P视频——11 参数的贝叶斯点估计（四）.mp4
12 参数的贝叶斯区间估计（一）.mp4
已存在第12P视频——12 参数的贝叶斯区间估计（一）.mp4
13 参数的贝叶斯区间估计（二）.mp4
已存在第13P视频——13 参数的贝叶斯区间估计（二）.mp4
14 可信区间与最大后验密度可信集（一）.mp4
已存在第14P视频——14 可信区间与最大后验密度可信集（一）.mp4
15 可信区间与最大后验密度可信集（二）.mp4
已存在第15P视频——15 可信区间与最大后验密度可信集（二）.mp4
16 可信区间与最大后验密度可信集（三）.mp4
已存在第16P视频——1

In [ ]:
import requests
from parsel import Selector
import json
import os


class GetBv():
    def __init__(self, bvid, page=1):
        '''
        bvid:视频号 eg:BV1hE411N7q2，str
        page: P几的视频，默认为1 int
        '''
        self.bvid = bvid
        self.page = page
        self.pg_dic, self.file_name, self.all_page = GetBv.get_name_pages(self)  # 视频名称表单、文件夹名称、总视频数
        # 表头
        self.headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:2.0.1) Gecko/20100101 Firefox/4.0.1',
                        'Referer': 'https://www.bilibili.com',}
    
    # 获取视频网址
    def get_video_audio_urls(self):
        url = f'https://www.bilibili.com/video/{self.bvid}?p={self.page}'
        r = requests.get(url)
        selector = Selector(text=r.text)
        # 视频连接
        video_audio = selector.xpath('/html/head/script[5]/text()').getall()
        video_dic = video_dic = json.loads(video_audio[0][video_audio[0].index('{'):])['data']['dash']['video'] # 找到{}才能loads
        video_url = video_dic[0]['baseUrl']
        # 音频连接
        audio = video_dic = json.loads(video_audio[0][video_audio[0].index('{'):])['data']['dash']['audio']
        audio_url = audio[0]['baseUrl']
        return video_url, audio_url
    
    # 获取视频名称表单并创建存放视频的文件夹
    def get_name_pages(self):
        # 获取名称表单
        url = f'https://www.bilibili.com/video/{self.bvid}'
        r_ng = requests.get(url)
        selector = Selector(r_ng.text)
        
        video_message = selector.xpath('/html/head/script[6]/text()').getall()
        video_message = json.loads(video_message[0][video_message[0].index('{'):video_message[0].index('};')+1])      # 找到{}才能loads
        
        all_page = video_message['videoData']['videos']    # 总页数/视频数
        page_name_dic = {}             # 页数对应的名称
        for x in video_message['videoData']['pages']:
            page_name_dic[x['page']] = x['part']
            
        # 创建文件夹
        file_name = video_message['videoData']['title']
        if os.path.exists(file_name) == False: # 判断是否存在该文件夹
            os.mkdir(file_name)
            
        return page_name_dic, file_name,all_page
        
    # 获取video.m4s和audio.m4s格式的音频和视频
    def get_mp4(self):
        # bv_name: 视频名称
        bv_name = self.pg_dic[self.page]
        path = f'{self.file_name}/{bv_name}'    # 视频路径
        if os.path.exists(f'{bv_name}.mp4'):  # 判断是否存在该视频
            print(f'已存在第{self.page}P视频——{bv_name}.mp4')
        else:
            video_url, audio_url = GetBv.get_video_audio_urls(self)
            from urllib import request
            # 为请求添加请求头
            opener = request.build_opener()
            request.install_opener(self.headersdersders)
            request.urlretrieve(video_url, f'{path}_video.m4s')
            request.urlretrieve(cudio_url,filename='{path}_audio.m4s')
            
            # 合并视频
            order = f'ffmpeg -i "{path}_video.m4s" -i "{path}_audio.m4s" -codec copy "{path}.mp4"'  # 加引号是为了解决路径不能包含空格 . 的问题
            os.system(order)
            # 删除原来的音频+视频.m4s文件
            os.remove(f"{path}_audio.m4s")
            os.remove(f"{path}_video.m4s")

            print(f'成功爬取,第{self.page}P视频——{bv_name}')

if __name__ == '__main__':
    bvid = 'BV1Pt411G7my'           # ----视频的id----
    getBv = GetBv(bvid)
    max_page = getBv.all_page        # 获取最大页数
    page_list = list(range(1, max_page+1))   # ---爬取第几P视频的列表-，记得加1--
    print(f'------------开始爬取------------\n 本次爬取第{page_list}P的视频')
    
    for page in page_list:
        getBv.page = page
        getBv.get_mp4()